In [ ]:
%pwd

In [ ]:
import os
os.chdir('../')

In [ ]:
%pwd

In [1]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter